In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
    master('local[2]').\
    appName('scholar').\
    getOrCreate()

sc = spark.sparkContext

gzfile = './data/*.gz'
sc_file = sc.textFile(gzfile)

data = spark.read\
    .option('compression', 'gzip')\
    .json(gzfile)\
    .select('id', 'title', 'year', 'entities', 'paperAbstract')\
    .write.save('./data/orc', format='orc', mode='append')

spark.stop()

In [2]:
from rake_nltk import Rake

from pyspark.sql import SparkSession
from pyspark import SQLContext
from pyspark.sql.types import ArrayType, StructType, StructField, FloatType, StringType
from pyspark.sql.functions import udf

In [1]:
!pip install rake_nltk

     |████████████████████████████████| 1.5MB 1.1MB/s eta 0:00:01
  ERROR: Complete output from command /opt/conda/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-46jk_i4s/rake-nltk/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-1limew8a --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/rake_nltk
  copying rake_nltk/__init__.py -> build/lib/rake_nltk
  copying rake_nltk/rake.py -> build/lib/rake_nltk
  copying rake_nltk/__version__.py -> build/lib/rake_nltk
  installing to build/bdist.linux-x86_64/wheel
  running install
  running install_lib
  creating build/bdist.linux-x86_64
  creating build/bdist.linux-x86_64/wheel
  creating build/bdist.linux-x86_64/wheel/rake_nltk
  copying build/lib/rake_nltk/__init__.py 

In [3]:
spark = SparkSession.builder.\
    master('local[2]').\
    appName('scholar').\
    getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)


data = sqlContext.read.format('orc').load('./data/orc')

rake_schema = ArrayType(StructType([
    StructField('score', FloatType()),
    StructField('concept', StringType())
]), containsNull=True)

In [ ]:
def apply_rake(text):
    r = Rake()
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases_with_scores()


spark_rake = udf(lambda text: apply_rake(text), rake_schema)

res = data.filter('paperAbstract!=\'\'').select('id', 'title', 'year', 'entities', spark_rake('paperAbstract').alias('candidateConcepts'))
print(res.count())

res.write.save('./data/candidates', format='orc', mode='append')

64594444
